# Data Model Mover

**This tutorial shows how to copy a data model from one team/ data pool to another one, independent of the cluster.**



**To do so we first need to connect to the source data model.**

In [9]:
from pycelonis import get_celonis

c_source = get_celonis("https://bbraun.eu-3.celonis.cloud", "MzJlM2FiNmQtODRhZS00OTQ0LTlkNTEtMzg3ZjdkOGFjZTFjOjFXNTNnaUJ6YVdDbkdlQUFTUTZsa0JNOER4dUJ4Q3pyanJPUXNONmFwQU1p")
source_data_model = c_source.datamodels.find("8fd104c3-e01e-44ea-a2dd-022f5d551854") 

2020-10-15 19:23:52 - pycelonis: Login successful! Hello Willy Thich


**In the next step we connect to the target data pool.**

In [10]:
c_target = get_celonis("https://bbraun.eu-3.celonis.cloud", "MzJlM2FiNmQtODRhZS00OTQ0LTlkNTEtMzg3ZjdkOGFjZTFjOjFXNTNnaUJ6YVdDbkdlQUFTUTZsa0JNOER4dUJ4Q3pyanJPUXNONmFwQU1p")
target_data_pool = c_target.pools.find("d1b99aa8-5ded-4410-b2bc-9659f9d53ba6")
data_source_name = ""

2020-10-15 19:23:59 - pycelonis: Login successful! Hello Willy Thich


**Create the data model and add the tables.**

In [11]:
if source_data_model.name in target_data_pool.datamodels.names:
    print('A data model with the same name does already exist in the target data pool. Please rename one.')
else: 
    target_data_model = target_data_pool.create_datamodel(source_data_model.name)                         # create target data model

    connection = target_data_pool.data_connections.names[data_source_name] if data_source_name else None  # choose the connection for the data model 

    target_data_model.add_tables_from_pool(source_data_model.tables, connection)                          # add the tables from the connection to the data model

**Copy Activity & Case table settings. If you copy to a data pool that does not contain the case and activity table in the specified connection, this step cannot be performed. Just continue with the next one.**

In [12]:
target_data_model.create_process_configuration(
            activity_table=source_data_model.process_configurations[0].activity_table.data["name"] if source_data_model.process_configurations[0].activity_table else None,
            case_table=source_data_model.process_configurations[0].case_table.data["name"] if source_data_model.process_configurations[0].case_table else None,
            case_column=source_data_model.process_configurations[0].case_column if source_data_model.process_configurations[0].activity_table else None,
            activity_column=source_data_model.process_configurations[0].activity_column if source_data_model.process_configurations[0].activity_table else None,
            timestamp_column=source_data_model.process_configurations[0].timestamp_column if source_data_model.process_configurations[0].activity_table else None,
            sorting_column=source_data_model.process_configurations[0].sorting_column if source_data_model.process_configurations[0].activity_table else None)

[<DatamodelProcessConfiguration, id b387e7c0-6b81-474a-9f0e-1dd267b0bebf, name Unnamed object with ID: b387e7c0-6b81-474a-9f0e-1dd267b0bebf>,]

**Add forgein key relationships.**

In [13]:
for fk in source_data_model.foreign_keys:
    target_data_model.create_foreign_key(fk["source_table"], fk["target_table"], fk["columns"], from_scratch=True)
print('Relationships copied.')

Relationships copied.


**Add table aliases.**

In [14]:
import pycelonis
for t in source_data_model.tables:
    if t.alias == t.name and isinstance(source_data_model, pycelonis.objects_ibc.Datamodel):
        target_t = target_data_model.tables.find(t.data["name"])
        try:
            target_t.alias = t.name
        except ValueError:
            pass
#    if t.alias != t.name and isinstance(source_data_model, pycelonis.objects_cpm4.Datamodel):
#        target_t = target_data_model.tables.find(t.data["name"])
#        try:
#            target_t.alias = t.alias
#        except ValueError:
#            pass
print("Please ignore the warnings. Congrats you copied the data model '"+ target_data_model.name + "'!")

Please ignore the warnings. Congrats you copied the data model 'SAP DEB - Warehouse Management Data Model'!
